<a href="https://colab.research.google.com/github/andykofman/Postura/blob/main/Postura_Live_Demo.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

**1) Get the project code**

In [2]:
# Clean workspace repo folder
!cd /content
!rm -rf Postura
!git clone https://github.com/andykofman/Postura.git

Cloning into 'Postura'...
remote: Enumerating objects: 86, done.
remote: Counting objects: 100% (45/45), done.
remote: Compressing objects: 100% (35/35), done.
remote: Total 86 (delta 12), reused 38 (delta 9), pack-reused 41 (from 1)
Receiving objects: 100% (86/86), 35.18 MiB | 28.91 MiB/s, done.
Resolving deltas: 100% (16/16), done.


**2) Change dir to Postura**

In [3]:
%cd /content/Postura

/content/Postura


**3) Remove Colab extras that can force protobuf>=5 (conflicts with MediaPipe)**

In [4]:
!pip uninstall -y ydf grpcio-status || true

Found existing installation: ydf 0.13.0
Uninstalling ydf-0.13.0:
  Successfully uninstalled ydf-0.13.0
Found existing installation: grpcio-status 1.71.2
Uninstalling grpcio-status-1.71.2:
  Successfully uninstalled grpcio-status-1.71.2


**4) Install pinned dependencies (deterministic)**

In [6]:
!python -V
!python -m pip install --upgrade pip
!pip install -r requirements.txt -c constraints.txt

Python 3.11.13
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.8/1.8 MB 19.1 MB/s eta 0:00:00
  Attempting uninstall: pip
    Found existing installation: pip 24.1.2
    Uninstalling pip-24.1.2:
      Successfully uninstalled pip-24.1.2
INFO: pip is looking at multiple versions of opencv-contrib-python to determine which version is compatible with other requirements. This could take a while.
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 18.3/18.3 MB 109.1 MB/s  0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 35.7/35.7 MB 46.3 MB/s  0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.9/49.9 MB 53.1 MB/s  0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 69.1/69.1 MB 56.2 MB/s  0:00:01
  Attempting uninstall: flatbuffers
    Found existing installation: flatbuffers 25.2.10
    Uninstalling flatbuffers-25.2.10:
      Successfully uninstalled flatbuffers-25.2.10
  Attempting uninstall: uvicorn
    Found existing installation: uvicorn 0.35.0
    Uninstalling uvicorn-0.35.0:
      Succe

**5) Hard-restart runtime so imports use the final environment**

if already restarted, skip to **step 6**

In [ ]:
import os, sys
os.kill(os.getpid(), 9)

**6) (After restart) Re-enter the repo**

In [6]:
%cd /content/Postura

/content/Postura


**7) Sanity checks (idempotent)**

In [7]:
!python scripts/colab_bootstrap.py --remove-colab-conflicts --run-mp-smoke

[env] Google Colab environment detected.
[python] Detected Python 3.11.13
[warn] Expected Python 3.10 for best compatibility.
[apt] All required packages present.
[colab] No conflicting preinstalled packages detected.
[pip] Installing/upgrading packages per requirements + constraints (idempotent).


[verify] Importing numpy, cv2, mediapipe...
2025-08-16 18:16:37.998444: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:467] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1755368198.057145    2460 cuda_dnn.cc:8579] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1755368198.075295    2460 cuda_blas.cc:1407] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
W0000 00:00:1755368198.124497    2460 computation_placer.cc:177] computation placer already 

**8) Start the API server in the background**

In [8]:
!pkill -f uvicorn || true
!nohup uvicorn api.main:app --host 0.0.0.0 --port 8000 --workers 1 > uvicorn.log 2>&1 &
!sleep 2 && sed -n '1,60p' uvicorn.log

^C
INFO:     Started server process [2712]
INFO:     Waiting for application startup.
INFO:     Application startup complete.
INFO:     Uvicorn running on http://0.0.0.0:8000 (Press CTRL+C to quit)


**9) Open ngrok tunnel (authenticated)**

Enter: **url/ui**


In [9]:
%pip -q install pyngrok
from pyngrok import ngrok
ngrok.kill()  # ensure clean slate
ngrok.set_auth_token("31Ma6WurjHpwLKZYddslQ16M8v5_7QcAgcGLyeZty1bZ7eupf")
tunnel = ngrok.connect(8000, "http", bind_tls=True)
public_url = tunnel.public_url
public_url, f"{public_url}/ui"

('https://a9e55acb4353.ngrok-free.app',
 'https://a9e55acb4353.ngrok-free.app/ui')